In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# -*- coding: utf-8 -*-
#torch imports
import torch
torch.set_printoptions(profile="full")
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
#some metrics
from sklearn.metrics import recall_score, precision_score, f1_score
#general imports
import os
import sys
sys.path.append('/content/gdrive/My Drive/chexpert_semantic_noise/noisy_data/baseline')
import h5py
import time
import json
import pickle
import shutil
import datetime
import threading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
# some local files
from tf_chexpert_utilities import *
from torch_chexpert_dataset import ChexpertDataset, MetricKeeper

# One Hit - Finish Concatenate

In [3]:
def concatenate_results():
  NEGATIVE_CASES = ['Enlarged Cardiomediastinum', 
                  'Cardiomegaly', 
                  'Lung Opacity', 
                  'Lung Lesion',
                  'Edema',
                  'Consolidation',
                  'Pneumonia',
                  'Atelectasis',
                  'Pneumothorax',
                  'Pleural Effusion',
                  'Pleural Other',
                  'Fracture',
                  'Support Devices']

  m_list = ['loss', 
            'acc', 
            'real_loss', 
            'real_acc', 
            'recall', 
            'precision', 
            'f_one_score',
            'positive_acc',
            'negative_acc',
            'positive_real_acc',
            'negative_real_acc']

  # finish metric keeper

  train_m_list = []
  val_m_list = []

  for m in m_list:
    train_m_list.append(f'train_{m}'.lower())
    val_m_list.append(f'val_{m}'.lower())
  for n in NEGATIVE_CASES:
    k = n.lower()
    k = k.replace(' ', '_').lower()
    train_m_list.append(f'train_{k}_acc')
    train_m_list.append(f'train_{k}_real_acc')
    val_m_list.append(f'val_{k}_acc')
    val_m_list.append(f'val_{k}_real_acc')

  print(train_m_list)
  print(val_m_list)

  source_folder = './network_training_predictions/torch_baseline'
  destination_folder = './network_training_predictions/torch_baseline/0'

  concatenated_metrics_e = {}
  concatenated_metrics_i = {}
  for m in train_m_list:
    te = f'{source_folder}/{m}_e'
    ti = f'{source_folder}/{m}_i'
    concatenated_metrics_e[te] = np.zeros(50)
    concatenated_metrics_i[ti] = np.zeros(419)

  for m in val_m_list:
    te = f'{source_folder}/{m}_e'
    ti = f'{source_folder}/{m}_i'
    concatenated_metrics_e[te] = np.zeros(50)
    concatenated_metrics_i[ti] = np.zeros(52)

  print('[TRAIN] concatenating individual results...')
  for m in train_m_list:
    t = f'{source_folder}/{m}_e'
    print(f'{t}...')
    try:
      for i in range(50):
        fname = f'{t}_{i}'
        concatenated_metrics_e[t][i] = unpickle(fname)
    except Exception as e:
      print(f'warning: {e}')
  print('[VAL] concatenating individual results...')
  for m in val_m_list:
    t = f'{source_folder}/{m}_e'
    print(f'{t}...')
    try:
      for i in range(50):
        fname = f'{t}_{i}'
        concatenated_metrics_e[t][i] = unpickle(fname)
    except Exception as e:
      print(f'warning: {e}')

  print('[TRAIN] saving concatenated results...')
  for m in train_m_list:
    source      = f'{source_folder}/{m}_e'
    destination = f'{destination_folder}/{m}_e'
    save_ndarray(destination, concatenated_metrics_e[source])
    print(f'saved from: {source} to: {destination}...')

  print('[VAL] saving concatenated results...')
  for m in val_m_list:
    source      = f'{source_folder}/{m}_e'
    destination = f'{destination_folder}/{m}_e'
    save_ndarray(destination, concatenated_metrics_e[source])
    print(f'saved from: {source} to: {destination}...')

  print('REMOVING UNNECESSARY FILES ...')
  x = os.listdir(source_folder)
  files = []
  for i in x:
    if 'train_' in i or 'val_' in i:
      files.append(i)
      
  for f in files:
    fname = f'{source_folder}/{f}'
    open(fname).close()
    os.remove(fname)